# Fitting 2D images with Gammapy

Gammapy does not have any special handling for 2D images, but treats them as a subset of maps. Thus, classical 2D image analysis can be done in 2 independent ways:- 
1. Using the sherpa pacakge, see: https://github.com/gammapy/gammapy/blob/master/tutorials/image_fitting_with_sherpa.ipynb\n",
2. Within gammapy, by assuming 2D analysis to be a sub-set of the generalised `maps`. Thus, analysis should proceed exactly as demonstrated in https://github.com/gammapy/gammapy/blob/master/tutorials/analysis_3d.ipynb, taking care of a few things that we mention in this tutorial
We consider 2D `images` to be a special case of 3D `maps`, ie, maps with only one energy bin. This is a major difference while analysing in `sherpa`, where the `maps` must not contain any energy axis.\n",
In this tutorial, we do a classical image analysis using three example observations of the Galactic center region with CTA - ie, study the source flux and morphology.



## Setup

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from gammapy.extern.pathlib import Path
from gammapy.data import DataStore
from gammapy.irf import EnergyDispersion, make_mean_psf, make_mean_edisp
from gammapy.maps import WcsGeom, MapAxis, Map, WcsNDMap
from gammapy.cube import MapMaker, MapEvaluator, PSFKernel, MapFit
from gammapy.cube.models import SkyModel
from gammapy.spectrum.models import PowerLaw2
from gammapy.image.models import SkyGaussian, SkyPointSource
from regions import CircleSkyRegion


## Prepare modeling input data

### The counts, exposure and the background maps
This is the same drill - use `DataStore` object to access the CTA observations and retrieve a list of observations by passing the observations IDs to the `.get_observations()` method, then use `MapMaker` to make the maps.

In [2]:
# Define which data to use and print some information
data_store = DataStore.from_dir("$GAMMAPY_DATA/cta-1dc/index/gps/")
data_store.info()
print(
    "Total observation time (hours): ",
    data_store.obs_table["ONTIME"].sum() / 3600,
)
print("Observation table: ", data_store.obs_table.colnames)

Data store:
HDU index table:
BASE_DIR: /Users/asinha/Gammapy-dev/gammapy-extra/datasets/cta-1dc/index/gps
Rows: 24
OBS_ID: 110380 -- 111630
HDU_TYPE: ['aeff', 'bkg', 'edisp', 'events', 'gti', 'psf']
HDU_CLASS: ['aeff_2d', 'bkg_3d', 'edisp_2d', 'events', 'gti', 'psf_3gauss']

Observation table:
Observatory name: 'N/A'
Number of observations: 4
Total observation time (hours):  2.0
Observation table:  ['OBS_ID', 'RA_PNT', 'DEC_PNT', 'GLON_PNT', 'GLAT_PNT', 'ZEN_PNT', 'ALT_PNT', 'AZ_PNT', 'ONTIME', 'LIVETIME', 'DEADC', 'TSTART', 'TSTOP', 'DATE-OBS', 'TIME-OBS', 'DATE-END', 'TIME-END', 'N_TELS', 'OBJECT', 'CALDB', 'IRF', 'EVENTS_FILENAME', 'EVENT_COUNT']


In [3]:
# Select some observations from these dataset by hand
obs_ids = [110380, 111140, 111159]
observations = data_store.get_observations(obs_ids)

In [4]:
emin, emax = [0.1, 10] * u.TeV
energy_axis = MapAxis.from_bounds(emin.value, emax.value, 10, 
            unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(0, 0),
    binsz=0.02,
    width=(10, 8),
    coordsys="GAL",
    proj="CAR",
    axes=[energy_axis],
)

Note that even when doing a 2D analysis, it is better to use fine energy bins in the beginning and then sum them over. This is to ensure that the background shape can be approximated by a power law function in each energy bin.

In [5]:
%%time
maker = MapMaker(geom, offset_max=4.0 * u.deg)
maps = maker.run(observations)

/Users/asinha/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


CPU times: user 11.6 s, sys: 1.48 s, total: 13.1 s
Wall time: 13.2 s


In [6]:
maps

{'background': WcsNDMap
 
 	geom  : WcsGeom 
  	axes  : lon, lat, energy
 	shape : (500, 400, 10)
 	ndim  : 3
 	unit  : '' 
 	dtype : float32 , 'counts': WcsNDMap
 
 	geom  : WcsGeom 
  	axes  : lon, lat, energy
 	shape : (500, 400, 10)
 	ndim  : 3
 	unit  : '' 
 	dtype : float32 , 'exposure': WcsNDMap
 
 	geom  : WcsGeom 
  	axes  : lon, lat, energy
 	shape : (500, 400, 10)
 	ndim  : 3
 	unit  : 'm2 s' 
 	dtype : float32 }

As we can see, the maps now have multiple bins in energy. We need to squash them to have one bin, and this can be done by specifying `keep_dims = True` while calling `make_images()`. This will compute a summed `counts` and `background` maps, and a spectral weighted `exposure`  map.

In [8]:
spectrum = PowerLaw2(index=2)
maps2D = maker.make_images(spectrum = spectrum, keepdims=True)

For a typical 2D analysis, using an energy dispersion usually does not make sense. A PSF map can be made as in the regular 3D case.

In [9]:
# mean PSF
geom2d = maps2D['exposure'].geom
src_pos = SkyCoord(0, 0, unit="deg", frame="galactic")
table_psf = make_mean_psf(observations, src_pos)

# PSF kernel used for the model convolution
psf_kernel = PSFKernel.from_table_psf(table_psf, geom2d, max_radius="0.3 deg")

Now, the analysis proceeds as usual. Just take care to use the proper geometry in this case.

## Define a mask

In [10]:
mask = Map.from_geom(geom2d)

region = CircleSkyRegion(center=src_pos, radius=0.6 * u.deg)
mask.data = mask.geom.region_mask([region])

## Modelling the source

This is the important thing to note in this analysis. Since modelling and fitting in `gammapy.maps` needs to have a combination of spectral models, we have to use a dummy Powerlaw as for the specrtal model and fix its index to 2. Since we are interested only in the integral flux, we will use the `PowerLaw2` model which directly fits an intergal flux.

In [11]:
spatial_model = SkyPointSource(lon_0="0.01 deg", lat_0="0.01 deg")
spectral_model = PowerLaw2(
    index=2.0, amplitude="3e-12 cm-2 s-1"
)
model = SkyModel(spatial_model=spatial_model, spectral_model=spectral_model)
model.parameters["index"].frozen = True

In [12]:
fit = MapFit(
    model=model,
    counts=maps2D["counts"],
    exposure=maps2D["exposure"],
    background=maps2D["background"],
    mask=mask,
    psf=psf_kernel,
)

In [13]:
%%time
result = fit.run(optimize_opts={"print_level": 1})

FCN = 468.27741980552673 
 TOTAL NCALL = 146 
 NCALLS = 146 
 
 
 EDM = 0.00030063004013465637 
 GOAL EDM = 1e-05 
 
 UP = 1.0 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,par_000_lon_0,-4.73178,0.186177,0,0,,,
2,par_001_lat_0,-5.39081,0.182312,0,0,,,
3,par_002_amplitude,23.3274,1.03168,0,0,,,
4,par_003_index,2,1,0,0,,,FIXED
5,par_004_emin,1,1,0,0,,,FIXED
6,par_005_emax,1,1,0,0,,,FIXED


CPU times: user 4.24 s, sys: 230 ms, total: 4.47 s
Wall time: 4.47 s


The output of the fit does not look very meaningful. To see the actual best-fit parameters, do a print on the result

In [14]:
print(result.model)

SkyModel

Parameters: 

	   name     value      error     unit   min max frozen
	--------- ---------- --------- -------- --- --- ------
	    lon_0 -4.732e-02 1.862e-03      deg nan nan  False
	    lat_0 -5.391e-02 1.823e-03      deg nan nan  False
	amplitude  2.333e-11 1.032e-12 cm-2 s-1 nan nan  False
	    index  2.000e+00 0.000e+00          nan nan   True
	     emin  1.000e-01 0.000e+00      TeV nan nan   True
	     emax  1.000e+02 0.000e+00      TeV nan nan   True

Covariance: 

	   name     lon_0     lat_0   amplitude   index      emin      emax  
	--------- --------- --------- --------- --------- --------- ---------
	    lon_0 3.466e-06 5.895e-08 5.567e-16 0.000e+00 0.000e+00 0.000e+00
	    lat_0 5.895e-08 3.324e-06 3.111e-16 0.000e+00 0.000e+00 0.000e+00
	amplitude 5.567e-16 3.111e-16 1.064e-24 0.000e+00 0.000e+00 0.000e+00
	    index 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
	     emin 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
	     emax 0.000

## Todo: Demonstrate plotting a flux map

## Exercises
1. Plot residual maps as done in the `analysis_3d` notebook
2. Iteratively add and fit sources as explained in `image_fitting_with_sherpa` notebook
